# Init

In [0]:
!pip install torch
!pip install pytorch_pretrained_bert

    100% |████████████████████████████████| 122kB 8.2MB/s 


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering, BertForPreTraining ,BertAdam

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Bert Generate One By One

In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM.from_pretrained(modelpath)

model.to('cuda')

example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#   for _ in range(512-len(indexed_tokens)):
#     indexed_tokens.append(0)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
#   for _ in range(512-len(loss_ids)):
#     loss_ids.append(-1)
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors
  print(tokenized_text,loss_ids,loss_ids[-1])
  print(len(indexed_tokens),len(loss_tensors))

100%|██████████| 382072689/382072689 [00:12<00:00, 30653301.52B/s]


['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, 2769] 2769
9 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, 3022] 3022
10 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1062] 1062
11 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6722] 6722
12 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 677] 677
13 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '上', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2119] 2119
14 1
['[CLS]', 'i', 'go', 'to', 'school', 'by', 'bus', '[SEP]', '我', '搭', '公', '車', '上', '學', '[MASK]'] [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 102]

In [0]:
print(example_pair)

{tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 2769]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,   103]],
       device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 3022]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
           103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 1062]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,   103]], device='cuda:0'): tensor([[  -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1, 6722]],
       device='cuda:0'), tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,  2769,  3022,
          1062,  6722,   103]], device='cuda:0'): tensor([[ -1,  -1,  -1,  -1,  -1

In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=5e-5,
                             warmup=0.1,
                             t_total=300)

In [0]:
model.train()
for i in range(0,60):
  eveloss = 0
  for k,v in example_pair.items():
    loss = model(k,masked_lm_labels=v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 44.37307119369507
step 1 : 24.271377563476562
step 2 : 12.52522087097168
step 3 : 22.794570922851562
step 4 : 54.069153785705566
step 5 : 26.595879077911377
step 6 : 17.794955253601074
step 7 : 19.63413095474243
step 8 : 14.428961277008057
step 9 : 17.612611770629883
step 10 : 3.119110107421875
step 11 : 0.34311676025390625
step 12 : 4.273902893066406
step 13 : 0.00643157958984375
step 14 : 0.012725830078125
step 15 : 0.002643585205078125
step 16 : 0.0011348724365234375
step 17 : 0.0010509490966796875
step 18 : 0.0018711090087890625
step 19 : 0.00031280517578125
step 20 : 0.000392913818359375
step 21 : 0.0003414154052734375
step 22 : 0.0001430511474609375
step 23 : 0.000186920166015625
step 24 : 0.00023651123046875
step 25 : 0.000225067138671875
step 26 : 9.72747802734375e-05
step 27 : 0.000118255615234375
step 28 : 0.000141143798828125
step 29 : 0.0001659393310546875
step 30 : 0.0002574920654296875
step 31 : 0.0001087188720703125
step 32 : 15.795350074768066
step 33 : 0.00021

In [0]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,len(predictions[0])-1]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
      break
    print(predicted_token)


['我']
['搭']
['公']
['車']
['上']
['學']


# Bert Generate One Time



In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
model = BertForMaskedLM.from_pretrained(modelpath)
model.to('cuda')

tokenized_text = tokenizer.tokenize(input_text)
for i in target_text:
  tokenized_text.append('[MASK]')
# tokenized_text.append('[SEP]')
for _ in range(128-len(tokenized_text)):
  tokenized_text.append('[MASK]')
# tokenized_text.append('[MASK]')
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')

loss_ids = []
loss_ids = [-1] * (len(tokenizer.tokenize(input_text)))
# loss_ids.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input_text)))
for i in target_text:
  loss_ids.append(tokenizer.convert_tokens_to_ids(i)[0])
loss_ids.append(tokenizer.convert_tokens_to_ids(['[SEP]'])[0])
for _ in range(128-len(loss_ids)):
  loss_ids.append(-1)
loss_tensors = torch.tensor([loss_ids]).to('cuda')

In [0]:
print(tokens_tensor,loss_tensors)
print(tokenizer.convert_ids_to_tokens(indexed_tokens))

tensor([[  101,   151,  8373,  8228,  9467,  8120, 10411,   102,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
           103,   103,   103,   103,   103,   103,  

In [0]:
# # Prepare optimizer
# param_optimizer = list(model.named_parameters())

# # hack to remove pooler, which is not used
# # thus it produce None grad that break apex
# param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.05},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.01}
#         ]
# optimizer = BertAdam(optimizer_grouped_parameters,
#                              lr=5e-5)

# optimizer = torch.optim.Adam(model.parameters(), lr=5e-7)
# optimizer = torch.optim.SGD(model.parameters(), lr = 5e-5, momentum=0.9)
optimizer = torch.optim.Adamax(model.parameters(), lr = 5e-5)

In [0]:
model.train()
for i in range(0,300):
  loss = model(tokens_tensor,masked_lm_labels=loss_tensors)
  eveloss = loss.mean().item()
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 7.913342475891113
step 1 : 7.833029270172119
step 2 : 7.197516441345215
step 3 : 6.325068473815918
step 4 : 5.70965576171875
step 5 : 5.5573506355285645
step 6 : 4.576362609863281
step 7 : 4.289361000061035
step 8 : 4.3420515060424805
step 9 : 4.592452526092529
step 10 : 4.3097662925720215
step 11 : 4.651139736175537
step 12 : 4.5207648277282715
step 13 : 4.477480411529541
step 14 : 4.2064900398254395
step 15 : 4.258938789367676
step 16 : 4.068099498748779
step 17 : 3.940248727798462
step 18 : 4.226219654083252
step 19 : 4.136264801025391
step 20 : 4.031257629394531
step 21 : 4.052146911621094
step 22 : 4.0309343338012695
step 23 : 3.9492135047912598
step 24 : 3.657867431640625
step 25 : 3.862555980682373
step 26 : 3.7650206089019775
step 27 : 3.5912110805511475
step 28 : 3.6331944465637207
step 29 : 3.9080066680908203
step 30 : 3.6006791591644287
step 31 : 3.616935968399048
step 32 : 3.6273269653320312
step 33 : 3.4186439514160156
step 34 : 3.361997365951538
step 35 : 3.23269

In [0]:
model.eval()
with torch.no_grad():
  predictions = model(tokens_tensor)
  start = len(tokenizer.tokenize(input_text))
  while start < len(predictions[0]):
    predicted_index = torch.argmax(predictions[0,start]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    if '[SEP]' in predicted_token:
        break
    print(predicted_token)
    start+=1

['我']
['搭']
['公']
['車']
['上']
['學']


# Bert Generate LSTM




In [0]:
input_text = "[CLS] I go to school by bus [SEP] "
target_text = "我搭公車上學"

In [0]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(modelpath)
example_pair = dict()

for i in range(0,len(target_text)+1):
  tokenized_text = tokenizer.tokenize(input_text)
  tokenized_text.extend(target_text[:i])
  tokenized_text.append('[MASK]')
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
  
  loss_ids = [-1] * (len(tokenizer.convert_tokens_to_ids(tokenized_text))-1)
  if i == len(target_text):
    loss_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))[0])
  else:
    loss_ids.append(tokenizer.convert_tokens_to_ids(target_text[i])[0])
  loss_tensors = torch.tensor([loss_ids]).to('cuda')
  
  example_pair[tokens_tensor] = loss_tensors

In [0]:
class BertLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        modelpath = "bert-base-chinese"
        self.bert = BertModel.from_pretrained(modelpath)
        self.rnn = nn.LSTM(num_layers=2,dropout=0.2, input_size=768, hidden_size=768//2)
        self.fc = nn.Linear(384, self.bert.config.vocab_size)
        
    def forward(self, x,y=None):
        
        self.bert.train()
        encoded_layers, _ = self.bert(x)
        for i in encoded_layers:
          enc, _ = self.rnn(i)
        logits = self.fc(enc)
        if y is not None:
          loss_fct  = nn.CrossEntropyLoss(ignore_index=-1)
          loss = loss_fct(logits.view(-1, logits.shape[-1]), y.view(-1))
          return loss
        return logits

In [0]:
model = BertLSTM()
optimizer = torch.optim.Adamax(model.parameters(), lr = 5e-5)

In [0]:
model.to('cuda')
model.train()
for i in range(0,150):
  eveloss = 0
  for k,v in example_pair.items():
    optimizer.zero_grad()
    loss = model(k,v)
    eveloss += loss.mean().item()
    loss.backward()
    optimizer.step()
  print("step "+ str(i) + " : " + str(eveloss))

step 0 : 69.75375080108643
step 1 : 69.47504615783691
step 2 : 69.33349609375
step 3 : 69.20606231689453
step 4 : 69.0837812423706
step 5 : 69.005859375
step 6 : 68.85943794250488
step 7 : 68.71227645874023
step 8 : 68.61512470245361
step 9 : 68.47988796234131
step 10 : 68.32582092285156
step 11 : 68.2575855255127
step 12 : 68.09355640411377
step 13 : 67.95422458648682
step 14 : 67.84716510772705
step 15 : 67.74326705932617
step 16 : 67.55989933013916
step 17 : 67.5284652709961
step 18 : 67.3663101196289
step 19 : 67.25604057312012
step 20 : 67.04909801483154
step 21 : 66.87121295928955
step 22 : 66.80055904388428
step 23 : 66.61228942871094
step 24 : 66.55533599853516
step 25 : 66.23289680480957
step 26 : 66.27696132659912
step 27 : 65.88841152191162
step 28 : 65.96708011627197
step 29 : 65.70754718780518
step 30 : 65.48617362976074
step 31 : 65.27843379974365
step 32 : 65.05558204650879
step 33 : 64.87465763092041
step 34 : 64.62165260314941
step 35 : 64.54038143157959
step 36 : 64.1

In [0]:
model.eval()
for k,v in example_pair.items():
    predictions = model(k)
    predicted_index = torch.argmax(predictions[0,-1]).item()
    if predicted_index < model.bert.config.vocab_size:
      predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

['我']
['搭']
['公']
['車']
['上']
['學']
['[SEP]']
